<h2>Mask propagation in the Functional API & Sequential API</h2>
<p>When we are using functional API and Sequential API, a mask generated by an embedding or masking layer will propogate through the network, for the layers that is capable of using them(example, RNN layers). Keras will automatically fetch the mask corresponding to an input and pass it to a layer taht knows how to use it. <br>
In this example, in the sequential model, the LSTM layer will automatiaclly receive a mask, whch means it will ignore padded values.</p>

In [3]:
#Importing required libraries
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
# Creating sequential model with mask_zero = True and LSTM layer of 32 units
model = keras.Sequential(
    [layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True), layers.LSTM(32),]
)

<p>In case of functional API with LSTM layer</p>

In [7]:
# Creating input layer
inputs = keras.Input(shape=(None,), dtype="int32")

#Creating embedding layer with mask_zero = True
x = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)(inputs)
outputs = layers.LSTM(32)(x)

#Creating the keras model with inputs and outputs
model = keras.Model(inputs, outputs)

<h2>Passing mask tensors directly to layers</h2>
<p>Layers taht can handle masks (example, LSTM layer) have a mask argument in their __call__ method. The layers that produce mask (example, Embedding) expose a compute_mask(input, previous_mask) method which we can call.<br>
Thus, you can pass the output of the compute_mask() method to the __call__ method, like below.</p>

In [9]:
#Creating a class MaskLayer by extending layers.Layer
class MaskLayer(layers.Layer):
    def __init__(self, **kwargs):
        super(MaskLayer, self).__init__(**kwargs)

        #Creating embedding layer
        self.embedding = layers.Embedding(input_dim=5000, output_dim=16, mask_zero=True)
        
        #creating lstm layer
        self.lstm = layers.LSTM(32)

    def call(self, inputs):
        x = self.embedding(inputs)
        
        # Note that you could also prepare a `mask` tensor manually.
        # It only needs to be a boolean tensor
        # with the right shape, i.e. (batch_size, timesteps).
        mask = self.embedding.compute_mask(inputs)
        
        output = self.lstm(x, mask=mask)  # The layer will ignore the masked values
        return output


layer = MaskLayer()
x = np.random.random((32, 10)) * 100
x = x.astype("int32")
layer(x)

<tf.Tensor: shape=(32, 32), dtype=float32, numpy=
array([[-0.00795619,  0.00524392,  0.00861745, ..., -0.00875466,
        -0.00633617, -0.00855093],
       [-0.00205287, -0.00309557,  0.00312118, ..., -0.00288415,
        -0.00807767,  0.00465092],
       [ 0.00791006,  0.00013945, -0.00069096, ...,  0.00251986,
         0.01511674, -0.00868041],
       ...,
       [ 0.00826469, -0.00043209,  0.00026146, ...,  0.01308176,
        -0.00833402,  0.0220686 ],
       [ 0.00078449,  0.00352711, -0.00116165, ..., -0.00150674,
        -0.01823967,  0.00790416],
       [-0.00200738, -0.00597855,  0.00148308, ..., -0.00708208,
        -0.00466533, -0.00834417]], dtype=float32)>